In [1]:
import cv2
import numpy as np
import pyttsx3
import threading
import mediapipe as mp
from ultralytics import YOLO

# Load YOLOv8 model
yolo_model = YOLO("yolov8n.pt")  # Or yolov8s.pt for better accuracy

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

# Threaded speak function
def speak(text):
    def _speak():
        engine = pyttsx3.init()
        engine.setProperty('rate', 150)
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Open video or webcam
cap = cv2.VideoCapture("video_example.mp4")  # Use 0 for webcam

# Store fall status
fall_detected_flags = {}

# Calculate angle between shoulder and hip midpoints
def get_angle(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    angle_rad = np.arctan2(y2 - y1, x2 - x1)
    return np.abs(np.degrees(angle_rad))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    results = yolo_model(frame)[0]

    for box in results.boxes:
        cls = int(box.cls[0])
        if cls != 0:
            continue  # Only detect 'person' class

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        person_crop = frame[y1:y2, x1:x2]
        rgb_crop = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
        pose_result = pose.process(rgb_crop)

        if pose_result.pose_landmarks:
            landmarks = pose_result.pose_landmarks.landmark

            ls = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * (x2 - x1) + x1,
                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * (y2 - y1) + y1]
            rs = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * (x2 - x1) + x1,
                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * (y2 - y1) + y1]
            lh = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * (x2 - x1) + x1,
                  landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * (y2 - y1) + y1]
            rh = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * (x2 - x1) + x1,
                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * (y2 - y1) + y1]

            mid_shoulder = [(ls[0] + rs[0]) / 2, (ls[1] + rs[1]) / 2]
            mid_hip = [(lh[0] + rh[0]) / 2, (lh[1] + rh[1]) / 2]

            angle = get_angle(mid_shoulder, mid_hip)
            person_id = f"{x1}-{y1}-{x2}-{y2}"

            if angle < 45:
                label = "FALL DOWN"
                color = (0, 0, 255)

                if not fall_detected_flags.get(person_id, False):
                    print("Fall detected! Speaking alert...")  # Debug
                    speak("Alert! A person has fallen down.")
                    fall_detected_flags[person_id] = True

                cv2.putText(frame, "ALERT: A person has FALLEN!", (30, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 3)
            else:
                label = "NOT FALLEN"
                color = (0, 255, 0)
                fall_detected_flags[person_id] = False

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            cv2.line(frame, tuple(np.int32(mid_shoulder)),
                     tuple(np.int32(mid_hip)), color, 2)

    cv2.imshow("Fall Detection", frame)
    if cv2.waitKey(10) & 0xFF == 27:  # ESC to quit
        break

cap.release()
cv2.destroyAllWindows()


100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:03<00:00, 1.77MB/s]



0: 384x640 2 persons, 689.5ms
Speed: 14.4ms preprocess, 689.5ms inference, 12.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 440.8ms
Speed: 4.0ms preprocess, 440.8ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 458.6ms
Speed: 6.2ms preprocess, 458.6ms inference, 4.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 53.3ms
Speed: 1.6ms preprocess, 53.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 52.1ms
Speed: 2.1ms preprocess, 52.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 49.7ms
Speed: 1.5ms preprocess, 49.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 52.0ms
Speed: 1.4ms preprocess, 52.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 45.7ms
Speed: 2.0ms preprocess, 45.7ms inference, 1.0ms postprocess per image at

Exception in thread Thread-8 (_speak):
Traceback (most recent call last):
  File "C:\Anaconda3\envs\dineshML\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\envs\dineshML\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Anaconda3\envs\dineshML\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\tiruk\AppData\Local\Temp\ipykernel_12472\3343273514.py", line 22, in _speak
  File "C:\Anaconda3\envs\dineshML\Lib\site-packages\pyttsx3\engine.py", line 219, in runAndWait
    raise RuntimeError(msg)
RuntimeError: run loop already started



0: 384x640 1 person, 1 dog, 46.6ms
Speed: 2.4ms preprocess, 46.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 43.2ms
Speed: 1.7ms preprocess, 43.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 frisbee, 50.0ms
Speed: 1.8ms preprocess, 50.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


Exception in thread Thread-9 (_speak):
Traceback (most recent call last):
  File "C:\Anaconda3\envs\dineshML\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\envs\dineshML\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Anaconda3\envs\dineshML\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\tiruk\AppData\Local\Temp\ipykernel_12472\3343273514.py", line 22, in _speak
  File "C:\Anaconda3\envs\dineshML\Lib\site-packages\pyttsx3\engine.py", line 219, in runAndWait
    raise RuntimeError(msg)
RuntimeError: run loop already started


...


0: 384x640 3 persons, 51.8ms
Speed: 1.3ms preprocess, 51.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Fall detected! Speaking alert...


Exception in thread Thread-60 (_speak):
Traceback (most recent call last):
  File "C:\Anaconda3\envs\dineshML\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\envs\dineshML\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Anaconda3\envs\dineshML\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\tiruk\AppData\Local\Temp\ipykernel_12472\3343273514.py", line 22, in _speak
  File "C:\Anaconda3\envs\dineshML\Lib\site-packages\pyttsx3\engine.py", line 219, in runAndWait
    raise RuntimeError(msg)
RuntimeError: run loop already started



0: 384x640 3 persons, 51.8ms
Speed: 1.6ms preprocess, 51.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 44.6ms
Speed: 2.2ms preprocess, 44.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 50.8ms
Speed: 1.3ms preprocess, 50.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


Exception in thread Thread-61 (_speak):
Traceback (most recent call last):
  File "C:\Anaconda3\envs\dineshML\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\envs\dineshML\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Anaconda3\envs\dineshML\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\tiruk\AppData\Local\Temp\ipykernel_12472\3343273514.py", line 22, in _speak
  File "C:\Anaconda3\envs\dineshML\Lib\site-packages\pyttsx3\engine.py", line 219, in runAndWait
    raise RuntimeError(msg)
RuntimeError: run loop already started


Fall detected! Speaking alert...

0: 384x640 2 persons, 52.2ms
Speed: 1.4ms preprocess, 52.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 frisbee, 49.9ms
Speed: 1.5ms preprocess, 49.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 frisbee, 50.2ms
Speed: 1.6ms preprocess, 50.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 frisbee, 41.3ms
Speed: 1.5ms preprocess, 41.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 frisbee, 1 skateboard, 46.2ms
Speed: 1.6ms preprocess, 46.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 48.2ms
Speed: 2.1ms preprocess, 48.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 frisbee, 50.0ms
Speed: 1.7ms preprocess, 50.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x6